In [101]:
%matplotlib inline 
import matplotlib.pyplot as plt 
import itertools 
import numpy as np 
import pandas 
from skbio import Protein 

In [102]:
with open( '../rosetta_runs/benchmark/bglb.pdb' ) as fn:
    lines = fn.readlines()
    atoms = [] 
    for i in lines:
        tokens = i.split() 
        if tokens[ 0 ] == 'ATOM' and tokens[ -1 ] is not 'H' and tokens[ 2 ] not in [ 'N', 'CA', 'O', 'C' ]:
            atoms.append( tokens ) 

In [103]:
cutoff = 5 
contacts = []
for atom1, atom2 in itertools.combinations( atoms, 2 ):
    atom1_index = int( atom1[ 5 ] ) - 1
    atom2_index = int( atom2[ 5 ] ) - 1
    if not atom1_index == atom2_index:
        distance = np.sqrt( np.sum( [ ( float( atom1[ i ] ) - float( atom2[ i ] ) ) ** 2 for i in [ 6, 7, 8 ] ] ) )
        if distance < cutoff:
            contacts.append( ( atom1_index, atom2_index ) ) 
        
my_contacts = set( contacts ) 

In [104]:
# "alignments" 

df = pandas.read_csv( '../data_sets/experimental/thermo_paper_data_set.csv', index_col=0 ) 
protein = Protein.read( '../data_sets/reference/bglb.pep' ) 

alignments = []
for mutant in df.index:
    if mutant != 'BglB':
        my_alignment = list( str( protein ) ) 
        my_sequence_pos = int( mutant[ 1:-1 ] )
        my_alignment[ my_sequence_pos - 1 ] = mutant[ -1 ] 
        alignments.append( my_alignment ) 

In [105]:
# glue 

alignment = zip( *alignments )
contacts = tuple( my_contacts ) 

In [106]:
# code from Phil Romero

In [107]:
from random import choice
import numpy
from itertools import product 

# # start with a family alignment (here I have 4 members, 10 positions)
# alignment = [
# ['A','A','P','F'],
# ['S','T','I','L'],
# ['Q','Q','Q','N'],
# ['F','T','F','T'],
# ['R','R','R','R'],
# ['S','K','K','R'],
# ['A','V','I','L'],
# ['V','H','K','D'],
# ['W','W','W','W'],
# ['I','L','M','F']]

# # contacts for the alignment
# contacts = ((0,4),(0,9),(1,2),(3,4),(5,8),(6,7),(7,9),(8,9)) # this could be up to N choose 2 possible contacts

In [108]:
# this list displays the amino acid options at every position in the alignment
# there could be up to 21 options per position
# this is something like the resfile for Rosetta design
solution_space = [sorted(set(pos)) for pos in alignment]

print( solution_space )

[['N'], ['T'], ['F'], ['I'], ['F'], ['P'], ['A'], ['T'], ['F'], ['M'], ['W'], ['G', 'N'], ['T'], ['A', 'S'], ['A', 'T'], ['A', 'N', 'S'], ['A', 'E', 'S'], ['A', 'Y'], ['A', 'C', 'P', 'Q', 'S'], ['I'], ['E'], ['G'], ['G'], ['T'], ['D'], ['E'], ['G'], ['G'], ['R'], ['T'], ['P'], ['L', 'S'], ['I'], ['A', 'W'], ['D'], ['T'], ['F'], ['C'], ['Q'], ['I'], ['P'], ['G'], ['K'], ['V'], ['I'], ['G'], ['G'], ['D'], ['C'], ['G'], ['D'], ['G', 'V'], ['A'], ['C'], ['D'], ['H'], ['F'], ['H'], ['H'], ['F'], ['K'], ['E'], ['D'], ['V'], ['Q'], ['L'], ['M'], ['K'], ['Q'], ['L'], ['G'], ['F', 'H'], ['L'], ['H'], ['Y'], ['A', 'R'], ['F'], ['S'], ['V'], ['A'], ['W'], ['P'], ['R'], ['I'], ['M'], ['P'], ['A'], ['A'], ['G'], ['I'], ['E', 'I'], ['N'], ['E'], ['E'], ['G'], ['L'], ['L'], ['F'], ['Y'], ['E'], ['H', 'R'], ['L'], ['L'], ['D'], ['E'], ['I'], ['E'], ['L'], ['A'], ['G'], ['L'], ['I'], ['P'], ['M'], ['L'], ['T'], ['L'], ['Y'], ['A', 'E', 'H', 'N'], ['A', 'F', 'H', 'W'], ['D', 'F'], ['L'], ['P'], ['Q'], [

In [109]:
# these are the regression parameters for the "Hamming kernel"
one_body_terms = []
for pos in range(len(solution_space)):
    for aa in solution_space[pos]:
        one_body_terms.append((pos,aa))

print( one_body_terms )

[(0, 'N'), (1, 'T'), (2, 'F'), (3, 'I'), (4, 'F'), (5, 'P'), (6, 'A'), (7, 'T'), (8, 'F'), (9, 'M'), (10, 'W'), (11, 'G'), (11, 'N'), (12, 'T'), (13, 'A'), (13, 'S'), (14, 'A'), (14, 'T'), (15, 'A'), (15, 'N'), (15, 'S'), (16, 'A'), (16, 'E'), (16, 'S'), (17, 'A'), (17, 'Y'), (18, 'A'), (18, 'C'), (18, 'P'), (18, 'Q'), (18, 'S'), (19, 'I'), (20, 'E'), (21, 'G'), (22, 'G'), (23, 'T'), (24, 'D'), (25, 'E'), (26, 'G'), (27, 'G'), (28, 'R'), (29, 'T'), (30, 'P'), (31, 'L'), (31, 'S'), (32, 'I'), (33, 'A'), (33, 'W'), (34, 'D'), (35, 'T'), (36, 'F'), (37, 'C'), (38, 'Q'), (39, 'I'), (40, 'P'), (41, 'G'), (42, 'K'), (43, 'V'), (44, 'I'), (45, 'G'), (46, 'G'), (47, 'D'), (48, 'C'), (49, 'G'), (50, 'D'), (51, 'G'), (51, 'V'), (52, 'A'), (53, 'C'), (54, 'D'), (55, 'H'), (56, 'F'), (57, 'H'), (58, 'H'), (59, 'F'), (60, 'K'), (61, 'E'), (62, 'D'), (63, 'V'), (64, 'Q'), (65, 'L'), (66, 'M'), (67, 'K'), (68, 'Q'), (69, 'L'), (70, 'G'), (71, 'F'), (71, 'H'), (72, 'L'), (73, 'H'), (74, 'Y'), (75, 'A'

In [110]:
# these are the regression parameters for the "structure-based kernel"
pairwise_terms = []
for pair in contacts:
    pos1 = pair[0]
    pos2 = pair[1]
    for aa1 in solution_space[pos1]:
        for aa2 in solution_space[pos2]:
            pairwise_terms.append(((pos1,aa1),(pos2,aa2)))

print( pairwise_terms )

[((14, 'A'), (74, 'Y')), ((14, 'T'), (74, 'Y')), ((172, 'Y'), (187, 'A')), ((248, 'A'), (252, 'F')), ((248, 'E'), (252, 'F')), ((156, 'N'), (157, 'W')), ((54, 'D'), (58, 'H')), ((80, 'W'), (125, 'I')), ((116, 'L'), (158, 'W')), ((160, 'T'), (216, 'I')), ((188, 'F'), (246, 'W')), ((220, 'A'), (342, 'F')), ((220, 'A'), (342, 'S')), ((220, 'M'), (342, 'F')), ((220, 'M'), (342, 'S')), ((331, 'F'), (381, 'A')), ((366, 'I'), (429, 'K')), ((403, 'A'), (421, 'Y')), ((403, 'C'), (421, 'Y')), ((403, 'N'), (421, 'Y')), ((18, 'A'), (19, 'I')), ((18, 'C'), (19, 'I')), ((18, 'P'), (19, 'I')), ((18, 'Q'), (19, 'I')), ((18, 'S'), (19, 'I')), ((355, 'A'), (371, 'R')), ((135, 'E'), (139, 'H')), ((399, 'A'), (403, 'A')), ((399, 'A'), (403, 'C')), ((399, 'A'), (403, 'N')), ((399, 'S'), (403, 'A')), ((399, 'S'), (403, 'C')), ((399, 'S'), (403, 'N')), ((347, 'P'), (393, 'K')), ((359, 'D'), (418, 'H')), ((238, 'R'), (310, 'V')), ((292, 'A'), (352, 'A')), ((292, 'A'), (352, 'E')), ((292, 'C'), (352, 'A')), ((

In [111]:
# lets generate 50 random sequences from our solution space
sequences = [''.join([choice(pos) for pos in solution_space]) for i in range(50)]
print( sequences )

['...']


In [112]:
# now we want to generate M by N feature matricies, where M is the number of sequences and N is the number of regression parameters
# these X matrices are binary and indicate the presence (=1) or absence (=0) of a particular interaction within a protein sequence

# X matrix for the one-body terms (for Hamming kernel)
X_one_body = []
for seq in sequences:
    x = [1 if seq[term[0]]==term[1] else 0 for term in one_body_terms] # if a sequence has a particular amino acid at a particular position, put a 1 at that term, else put a 0
    X_one_body.append(x)


In [113]:
# X matrix for the pairwise terms (for str-based kernel)
X_pairwise = []
for seq in sequences:
    x = [1 if (seq[term[0][0]]==term[0][1] and seq[term[1][0]]==term[1][1]) else 0 for term in pairwise_terms] # if a sequence has a particular amino acid pair at a particular contact, put a 1 at that term, else put a 0
    X_pairwise.append(x)


print( X_pairwise )

[[0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0,

In [114]:
# these X matrices can then be used as inputs for any type of regression or learning algorithm

# we can generate the kernel functions:
X = numpy.array(X_one_body)
K_hamming = numpy.dot(X,X.T)

print( X.shape )
print( K_hamming.shape )

(50, 573)
(50, 50)


In [115]:
X = numpy.array(X_pairwise)
K_structure = numpy.dot(X,X.T)

print( X.shape )
print(  K_structure.shape ) 

(50, 2691)
(50, 50)
